# Анализ текстов

Данное задание основано на материалах лекций по методу опорных векторов.

## Вы научитесь:

• находить оптимальные параметры для метода опорных векторов

• работать с текстовыми данными

## Введение

Метод опорных векторов (Support Vector Machine, SVM) — один из видов линейных классификаторов. Функционал, который он оптимизирует, направлен на максимизацию ширины разделяющей полосы между классами. Из теории статистического обучения известно, что эта ширина тесно связана с обобщающей способностью алгоритма, а ее максимизация позволяет бороться с переобучением.

Одна из причин популярности линейных методов заключается в том, что они хорошо работают на разреженных данных. Так называются выборки с большим количеством признаков, где на каждом объекте большинство признаков равны нулю. Разреженные данные возникают, например, при работе с текстами. Дело в том, что текст удобно кодировать с помощью "мешка слов" — формируется столько признаков, сколько всего уникальных слов встречается в текстах, и значение каждого признака равно числу вхождений в документ соответствующего слова. Ясно, что общее число различных слов в наборе текстов может достигать десятков тысяч, и при этом лишь небольшая их часть будет встречаться в одном конкретном тексте.
Можно кодировать тексты хитрее, и записывать не количество вхождений слова в текст, а TF-IDF. Это показатель, который равен произведению двух чисел: TF (term frequency) и IDF (inverse document frequency). Первая равна отношению числа вхождений слова в документ к общей длине документа. Вторая величина зависит от того, в скольки документах выборки встречается это слово. Чем больше таких документов, тем меньше IDF. Таким образом, TF-IDF будет иметь высокое значение для тех слов, которые много раз встречаются в данном документе, и редко встречаются в остальных.

## Реализация в SciKit Learn:

Загрузим объекты из новостного датасета 20 newsgroups, относящиеся к категориям "космос" и "атеизм".

In [1]:
from sklearn import datasets

newsgroups = datasets.fetch_20newsgroups(subset='all',categories=['alt.atheism', 'sci.space'])
target = newsgroups.target
data = newsgroups.data

Вычислим TF-IDF-признаки для всех текстов:

In [2]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
data_fit = vectorizer.fit_transform(data).toarray()
words=vectorizer.get_feature_names_out()

Поберём минимальный лучший параметр C из множества [10^-5, 10^-4, ... 10^4, 10^5] для SVM с линейным ядром (kernel='linear'):

In [3]:
from sklearn.svm import SVC
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
import numpy as np
grid = {'C': np.power(10.0, np.arange(-5, 6))}
cv = KFold(n_splits=5, shuffle=True, random_state=241)
clf = SVC(kernel='linear', random_state=241)
gs = GridSearchCV(clf, grid, scoring='accuracy', cv=cv)
gs.fit(data_fit, target) #ДОЛГО РАБОТАЕТ

GridSearchCV(cv=KFold(n_splits=5, random_state=241, shuffle=True),
             estimator=SVC(kernel='linear', random_state=241),
             param_grid={'C': array([1.e-05, 1.e-04, 1.e-03, 1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02,
       1.e+03, 1.e+04, 1.e+05])},
             scoring='accuracy')

Обучим SVM по всей выборке с оптимальным параметром C:

In [4]:
C=gs.best_estimator_.C
clf = SVC(C=C,kernel='linear', random_state=241)
result=clf.fit(data_fit, target)

Найдём 10 слов с наибольшим абсолютным значением веса:

In [12]:
import pandas as pd
weights = [abs(element[0]) for element in result.coef_.T]
combine = dict(zip(words,weights))
words_df = pd.DataFrame(data=combine,index=[0]).transpose()
words_df.columns = ['weights']
words_df.sort_values(['weights'],ascending=False,inplace = True)
best_words_df = pd.DataFrame(data=words_df.head(10).index.values)
print("10 наиболее весомых слов:",','.join(best_words_df[0].to_numpy()))

10 наиболее весомых слов: space,god,atheism,atheists,moon,sky,religion,bible,keith,sci
